In [8]:
import torch
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [9]:
# Set prompt template for generation (optional)
from llama_index.core import PromptTemplate

def completion_to_prompt(completion):
   return f"<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{completion}<|im_end|>\n<|im_start|>assistant\n"

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|im_start|>system\n{message.content}<|im_end|>\n"
        elif message.role == "user":
            prompt += f"<|im_start|>user\n{message.content}<|im_end|>\n"
        elif message.role == "assistant":
            prompt += f"<|im_start|>assistant\n{message.content}<|im_end|>\n"

    if not prompt.startswith("<|im_start|>system"):
        prompt = "<|im_start|>system\n" + prompt

    prompt = prompt + "<|im_start|>assistant\n"

    return prompt

# Set Qwen2.5 as the language model and set generation config
Settings.llm = HuggingFaceLLM(
    model_name="Qwen/Qwen2.5-0.5B",
    tokenizer_name="Qwen/Qwen2.5-0.5B",
    context_window=30000,
    max_new_tokens=2000,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)

# Set embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "all-MiniLM-L6-v2"
)

# Set the size of the text chunk for retrieval
Settings.transformations = [SentenceSplitter(chunk_size=512)]

/home/drissdo/anaconda3/envs/thesis/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# documents = SimpleDirectoryReader("../data").load_data()
# index = VectorStoreIndex.from_documents(
#     documents,
#     embed_model = Settings.embed_model,
#     trasformations = Settings.transformations
# )

In [11]:
import chromadb
# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart") 

In [12]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb  

# Assuming 'chroma_collection' is defined elsewhere in your code
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader("../data").load_data()
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
    transformations=Settings.transformations,
    storage_context=storage_context
)


In [13]:
query_engine = index.as_query_engine()
your_query = "Introduction about university?"
print(query_engine.query(your_query).response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Context information is below.
---------------------
file_path: /home/drissdo/Desktop/Work/Educational-Advisor-Chatbot/src/../data/tuyen_sinh_en.json

[
  {"text":"The International University, established in December 2003, is the only multidisciplinary public university in Vietnam that teaches and conducts research entirely in English. The university offers undergraduate and postgraduate programs, focusing on economics, management, and engineering technology. With a model that meets international standards, the university collaborates with prestigious universities from the United States, Europe, and the Asia-Pacific region. The university's goal is to become a leading research university in Vietnam and the region, providing high-quality human resources to meet the demands of integration.\n\nThe International University offers 23 training programs granted by the International University itself, and 20 training programs in affiliation with prestigious partners.\n\nThe university offers a